## DEBUG

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import logging

from pkg import c, m, f 

# classes 
paths = c.PathManager()
data = c.PeakImageDataset(paths)
prep = c.DataPreparation(paths, data, batch_size=5)
water_h5 = data.load_h5(paths.water_background_h5)
ip = c.ImageProcessor(water_h5)
p = c.PeakThresholdProcessor(threshold_value=10)

Number of peak images: 20
Number of water images: 20
Number of label images: 20
<class 'numpy.ndarray'>


In [2]:
# if not already generated 
# ip.process_directory(paths, p.threshold_value) 

In [3]:

# model 
model = m.ResNet50BraggPeakClassifier()
# loss function/ combines a Sigmoid layer and the BCELoss in one single class
criterion = nn.BCEWithLogitsLoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print("Criterion: ", criterion)
print("Optimizer: \n", optimizer)
print("Learning rate: ", optimizer.param_groups[0]['lr'])
# data loaders 
train_loader, test_loader = prep.prep_data()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/eseverett/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 24.9MB/s]


Criterion:  BCEWithLogitsLoss()
Optimizer: 
 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
Learning rate:  0.001
Data prepared.
Train size: 16
Test size: 4
Batch size: 5
Number of batches in train_loader: 4 




In [4]:
"""Training the model"""
num_epochs = 1 
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
train_losses = []
test_losses = []

logging.info('Staring training...')
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader: 
        peak_images, water_images = inputs
        # zero parameter gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(peak_images)
        loss = criterion(outputs, labels)
        # backward pass/optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    logging.info(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

2024-03-14 19:53:46,590 - INFO - Staring training...
